In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import os
import custom_graphgym  # noqa, register custom modules
import torch
from torch_geometric import seed_everything
from torch_geometric.graphgym.cmd_args import parse_args
from torch_geometric.graphgym.config import (
    cfg,
    dump_cfg,
    load_cfg,
    set_out_dir,
    set_run_dir,
)
from torch_geometric.graphgym.logger import set_printing
from torch_geometric.graphgym.model_builder import create_model
from torch_geometric.graphgym.train import GraphGymDataModule, train
from torch_geometric.graphgym.utils.agg_runs import agg_runs
from torch_geometric.graphgym.utils.comp_budget import params_count
from torch_geometric.graphgym.utils.device import auto_select_device

In [3]:
from torch_geometric.graphgym import register

In [4]:
register.head_dict.keys()

dict_keys(['node', 'link_pred', 'edge', 'graph', 'head', 'custom'])

In [5]:
import argparse

class NotebookArgParser:
    def __init__(self, args_str):
        self.args = self.parse_args(args_str)

    def parse_args(self, args_str):
        parser = argparse.ArgumentParser(description='GraphGym')

        # Add command-line arguments
        parser.add_argument('--cfg',
                            dest='cfg_file',
                            type=str,
                            required=True,
                            help='The configuration file path.')
        parser.add_argument('--repeat',
                            type=int,
                            default=1,
                            help='The number of repeated jobs.')
        parser.add_argument('--mark_done',
                            action='store_true',
                            help='Mark yaml as done after a job has finished.')
        parser.add_argument('opts',
                            default=None,
                            nargs=argparse.REMAINDER,
                            help='See graphgym/config.py for remaining options.')

        # Parse the command-line arguments
        args = parser.parse_args(args_str)

        return args

In [6]:
# Emulate command-line arguments using input cells
command = "python main_pyg.py --cfg ./configs/pyg/example_link_debug.yaml --repeat 3"
args_str = command.split()[2:]
args_str


['--cfg', './configs/pyg/example_link_debug.yaml', '--repeat', '3']

In [7]:
# Create a NotebookArgParser instance
notebook_parser = NotebookArgParser(args_str)

# Access parsed arguments
args = notebook_parser.args
print("Parsed Arguments:")
print(f"Configuration File: {args.cfg_file}")
print(f"Repeat: {args.repeat}")
print(f"Mark Done: {args.mark_done}")
print(f"Remaining Options: {args.opts}")

Parsed Arguments:
Configuration File: ./configs/pyg/example_link_debug.yaml
Repeat: 3
Mark Done: False
Remaining Options: []


In [8]:
# Load config file
load_cfg(cfg, args)
set_out_dir(cfg.out_dir, args.cfg_file)
# Set Pytorch environment
torch.set_num_threads(cfg.num_threads)
dump_cfg(cfg)

for i in range(args.repeat):
    set_run_dir(cfg.out_dir, i)
    set_printing()
    # Set configurations for each run
    cfg.seed = cfg.seed + 1
    seed_everything(cfg.seed)
    auto_select_device() # if not set in the yaml config, set to cuda accelerator if available and single device
    # Set machine learning pipeline
    datamodule = register.train_dict["CustomGraphGymDataModule"]()
    cfg.share.dim_out = 1 # TODO fix this bug, that happend in set_dataset_info because dataset._data.y might have node labels (not edge labels)
    cfg.share.num_splits = 3 # TODO fix this bug
    model = create_model()
    # Print model info
    logging.info(model)
    logging.info(cfg)
    cfg.params = params_count(model)
    logging.info('Num parameters: %s', cfg.params)
    # Call the custom training function
    register.train_dict["train_pl"](model, datamodule, logger=True)
    #train(model, datamodule, logger=True)

DOWNLOADING CUSTOM DATASET LOADER
INITIALIZING NEW CUSTOM EDGE HEAD


/home/stu13/s18/lb9849/BioCompute/pytorch_geometric/torch_geometric/data/in_memory_dataset.py:301: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


GraphGymModule(
  (model): GNN(
    (encoder): FeatureEncoder()
    (mp): GNNStackStage(
      (layer0): GeneralLayer(
        (layer): GCNConv(
          (model): GCNConv(1433, 64)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
      (layer1): GeneralLayer(
        (layer): GCNConv(
          (model): GCNConv(64, 64)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (post_mp): ExampleGNNEdgeHead(
      (layer_post_mp): MLP(
        (model): Sequential(
          (0): Linear(
            (model): Linear(64, 64, bias=True)
          )
        )
      )
    )
  )
)
accelerator: cuda
benchmark: False
bn:
  eps: 1e-05
  mom: 0.1
cfg_dest: config.yaml
custom_metrics: []
dataset:
  cache_load: False
  cache_save: False
  dir: ./datasets
  edge_dim: 128
  edge_encoder: False
  edge_encoder_bn: True
  edge_encoder_name: Bond
  edge_message_ratio: 0.8
  edge_negative_sampling_ratio: 1.0
  edge_train_mode: all
  enc

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: results/link_predict_debug/example_link_debug/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]

  | Name  | Type | Params
-------------------------------
0 | model | GNN  | 100 K 
-------------------------------
100 K     Trainable params
0         Non-trainable params
100 K     Total params
0.400     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  8.41it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  2.71it/s, v_num=2]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]
/home/stu13/s18/lb9849/miniconda3/envs/pyg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 31.84it/s] 


In [ ]:
#cfg.device = '-1'

### Previous working code

In [ ]:
# Load config file
load_cfg(cfg, args)
set_out_dir(cfg.out_dir, args.cfg_file)

# Set Pytorch environment
torch.set_num_threads(cfg.num_threads)
dump_cfg(cfg)
# Repeat for different random seeds

In [ ]:
set_run_dir(cfg.out_dir)
set_printing()
# Set configurations for each run
cfg.seed = cfg.seed + 1
seed_everything(cfg.seed)
auto_select_device()
# Set machine learning pipeline
# Load the custom GraphGymDataModule

In [ ]:
cfg.device = 8
cfg.accelerator = 'cuda'

In [ ]:
cfg.run_dir

In [ ]:
datamodule = register.train_dict["CustomGraphGymDataModule"]()

In [ ]:
datamodule.dataset

In [ ]:
datamodule._train_dataloader

In [ ]:
datamodule.dataset.num_edges

In [ ]:
cfg.train.batch_size = datamodule.dataset.num_edges

In [ ]:
for b in datamodule._train_dataloader:
    print(b)

In [ ]:
cfg.share.dim_in

In [ ]:
cfg.devices = 8

In [ ]:
cfg.accelerator = 'cuda:8'

In [ ]:
cfg.share.dim_out = 1 # TODO fix this bug, that happend in set_dataset_info because dataset._data.y might have node labels (not edge labels)

In [ ]:
model = create_model()
# Print model infos
logging.info(model)
logging.info(cfg)
cfg.params = params_count(model)
logging.info("Num parameters: %s", cfg.params)

In [ ]:
# for b in datamodule.train_dataloader():
#     print(b)


In [ ]:
# b = next(iter(datamodule.test_dataloader()))

In [ ]:
# from torch_geometric.loader import LinkNeighborLoader

# lb = LinkNeighborLoader(data=datamodule.splits['train'],
#             num_neighbors=[-1],
#             batch_size=8448,
#             edge_label_index= datamodule.splits['train'].edge_label_index,
#             edge_label= datamodule.splits['train'].edge_label,
#             shuffle=True,
#             num_workers=cfg.num_workers,
#             pin_memory=True
#         )

In [ ]:
# for b in lb:
#     print(b)

In [ ]:
# datamodule.splits['test'].edge_label

In [ ]:
cfg.share.num_splits = 3 # TODO fixx thisss!

In [ ]:
# Call the custom training function
register.train_dict["train_pl"](model, datamodule, logger=True)